# info 뜯어보기

file은 verilator를 통해서 읽어온 뒤에 xmltodict로 변환시켜놓았다.


In [1]:
import sys

from pprint import pprint

import json
import xmltodict

with open("./info.xml","r") as f:
    xmlString = f.read()

info = xmltodict.parse(xmlString)

In [2]:
len(info)

1

In [3]:
info.keys()

odict_keys(['verilator_xml'])

## dictionary 구조

```
'verilator_xml'
|
|-files
|-module_files
|-cells
|-netlist

```

위와 같이 총 4개의 key가 존재 하는 dictionary로 되어 있는대 

각 key에 상응하는 value 값 역시 dictionary 입니다. 

In [4]:
info['verilator_xml'].keys()

odict_keys(['files', 'module_files', 'cells', 'netlist'])

### files의 구조

files는  
file아래에  

id, filename, language로 이루어져 있습니다

In [5]:
info['verilator_xml']['files']['file']

[OrderedDict([('@id', 'a'),
              ('@filename', '<built-in>'),
              ('@language', '1800-2017')]),
 OrderedDict([('@id', 'b'),
              ('@filename', '<command-line>'),
              ('@language', '1800-2017')]),
 OrderedDict([('@id', 'c'),
              ('@filename', 'connection_test.v'),
              ('@language', '1800-2017')])]

In [6]:
val = info['verilator_xml']['files']['file']

### f_name 변수 생성

추후에 사용할 가능성이 있어서 f_name변수를 생성 했습니다. 

In [7]:
f_name={}
for i in val:
    f_name[i['@id']] = i["@filename"]
    if(i['@id'] =='c'):
        print(i["@filename"])

connection_test.v


In [8]:
f_name

{'a': '<built-in>', 'b': '<command-line>', 'c': 'connection_test.v'}

## file 읽어서 저장해두기

In [9]:
code={}
with open(f_name['c'],"r") as f:
    file = f.read()

code['c'] = file.split("\n")

------

## module_files 분석

verilator_xml가 가진 두번째 값  

module_files를 분석해보겠습니다. 


In [10]:
for i in info['verilator_xml'].keys():
    print(i)
    pprint(type(info['verilator_xml'][i]))

files
<class 'collections.OrderedDict'>
module_files
<class 'collections.OrderedDict'>
cells
<class 'collections.OrderedDict'>
netlist
<class 'collections.OrderedDict'>


In [11]:
info['verilator_xml']['module_files'].keys()

odict_keys(['file'])

In [12]:
info['verilator_xml']['module_files']['file'].keys()

odict_keys(['@id', '@filename', '@language'])

In [13]:
info['verilator_xml']['module_files']['file']

OrderedDict([('@id', 'c'),
             ('@filename', 'connection_test.v'),
             ('@language', '1800-2017')])

## module_files는?

module에 대한 정보가 담긴 file이 무엇인지 적혀 있는것 같습니다. 

앞서 f_name으로 분류 했지만 여기서 그냥 f_name을 만드는게 더 수월할것 같아서 f_name은 변경하도록 하겠습니다. 


In [14]:
val = info['verilator_xml']['module_files']['file']

f_name={}

#하나면 그냥 넣고 여러개면 여러번 넣기

try:
    f_name[val['@id']] = val['@filename']
except:
    for i in val:
        f_name[i['@id']] = i["@filename"]

In [15]:
f_name

{'c': 'connection_test.v'}

## cells는?

In [16]:
for i in info['verilator_xml'].keys():
    print(i)
    pprint(type(info['verilator_xml'][i]))

files
<class 'collections.OrderedDict'>
module_files
<class 'collections.OrderedDict'>
cells
<class 'collections.OrderedDict'>
netlist
<class 'collections.OrderedDict'>


In [17]:
info['verilator_xml']['cells'].keys()

odict_keys(['cell'])

In [18]:
info['verilator_xml']['cells']['cell'].keys()

odict_keys(['@fl', '@loc', '@name', '@submodname', '@hier', 'cell'])

### cells 의 구성

```
cells
|- '@fl',  
|- '@loc',  
|- '@name',  
|- '@submodname',  
|- '@hier',  
|- 'cell'
```

In [19]:
for i in info['verilator_xml']['cells']['cell'].keys():
    print(i," : ",info['verilator_xml']['cells']['cell'][i])

@fl  :  c1
@loc  :  c,1,8,1,23
@name  :  connection_test
@submodname  :  connection_test
@hier  :  connection_test
cell  :  [OrderedDict([('@fl', 'c20'), ('@loc', 'c,20,8,20,18'), ('@name', 'u_master_0'), ('@submodname', 'master'), ('@hier', 'connection_test.u_master_0')]), OrderedDict([('@fl', 'c37'), ('@loc', 'c,37,8,37,18'), ('@name', 'u_master_1'), ('@submodname', 'master'), ('@hier', 'connection_test.u_master_1')]), OrderedDict([('@fl', 'c46'), ('@loc', 'c,46,7,46,16'), ('@name', 'u_slave_1'), ('@submodname', 'slave'), ('@hier', 'connection_test.u_slave_1')]), OrderedDict([('@fl', 'c63'), ('@loc', 'c,63,8,63,18'), ('@name', 'u_master_2'), ('@submodname', 'master'), ('@hier', 'connection_test.u_master_2')]), OrderedDict([('@fl', 'c72'), ('@loc', 'c,72,7,72,16'), ('@name', 'u_slave_2'), ('@submodname', 'slave'), ('@hier', 'connection_test.u_slave_2')]), OrderedDict([('@fl', 'c85'), ('@loc', 'c,85,8,85,18'), ('@name', 'u_master_3'), ('@submodname', 'master'), ('@hier', 'connection_te

In [20]:
pprint(info['verilator_xml']['cells']['cell']['cell'])

[OrderedDict([('@fl', 'c20'),
              ('@loc', 'c,20,8,20,18'),
              ('@name', 'u_master_0'),
              ('@submodname', 'master'),
              ('@hier', 'connection_test.u_master_0')]),
 OrderedDict([('@fl', 'c37'),
              ('@loc', 'c,37,8,37,18'),
              ('@name', 'u_master_1'),
              ('@submodname', 'master'),
              ('@hier', 'connection_test.u_master_1')]),
 OrderedDict([('@fl', 'c46'),
              ('@loc', 'c,46,7,46,16'),
              ('@name', 'u_slave_1'),
              ('@submodname', 'slave'),
              ('@hier', 'connection_test.u_slave_1')]),
 OrderedDict([('@fl', 'c63'),
              ('@loc', 'c,63,8,63,18'),
              ('@name', 'u_master_2'),
              ('@submodname', 'master'),
              ('@hier', 'connection_test.u_master_2')]),
 OrderedDict([('@fl', 'c72'),
              ('@loc', 'c,72,7,72,16'),
              ('@name', 'u_slave_2'),
              ('@submodname', 'slave'),
              ('@hier', 'co

In [21]:
for i in info['verilator_xml']['cells']['cell']['cell']:
    print(i['@name'],"\t//\t",i['@submodname'], ' \t//\t', i['@hier'])

u_master_0 	//	 master  	//	 connection_test.u_master_0
u_master_1 	//	 master  	//	 connection_test.u_master_1
u_slave_1 	//	 slave  	//	 connection_test.u_slave_1
u_master_2 	//	 master  	//	 connection_test.u_master_2
u_slave_2 	//	 slave  	//	 connection_test.u_slave_2
u_master_3 	//	 master  	//	 connection_test.u_master_3
u_slave_3 	//	 slave  	//	 connection_test.u_slave_3


## netlist 확인

In [22]:
for i in info['verilator_xml'].keys():
    print(i)
    pprint(type(info['verilator_xml'][i]))

files
<class 'collections.OrderedDict'>
module_files
<class 'collections.OrderedDict'>
cells
<class 'collections.OrderedDict'>
netlist
<class 'collections.OrderedDict'>


In [23]:
info['verilator_xml']['netlist'].keys()

odict_keys(['module', 'typetable'])

### netlist는

module과 typetable이 있네요  



In [24]:
info['verilator_xml']['netlist']['module']

[OrderedDict([('@fl', 'c1'),
              ('@loc', 'c,1,8,1,23'),
              ('@name', 'connection_test'),
              ('@origName', 'connection_test'),
              ('@topModule', '1'),
              ('var',
               [OrderedDict([('@fl', 'c2'),
                             ('@loc', 'c,2,16,2,23'),
                             ('@name', 'i_clock'),
                             ('@dtype_id', '1'),
                             ('@dir', 'input'),
                             ('@vartype', 'logic'),
                             ('@origName', 'i_clock')]),
                OrderedDict([('@fl', 'c3'),
                             ('@loc', 'c,3,16,3,24'),
                             ('@name', 'i_resetn'),
                             ('@dtype_id', '1'),
                             ('@dir', 'input'),
                             ('@vartype', 'logic'),
                             ('@origName', 'i_resetn')]),
                OrderedDict([('@fl', 'c4'),
                            

In [25]:
type(info['verilator_xml']['netlist']['module'])

list

In [26]:
len(info['verilator_xml']['netlist']['module'])

3

## module 길이가 3인 이유

현재 읽어 들여온 code의 module은 총 3개 입니다. 

connection_test

master  

slave  

위 3가지가 각각 모듈이기 때문에 

나눠서 봅니다.

In [27]:
for i in info['verilator_xml']['netlist']['module']:
    print(i.keys())

odict_keys(['@fl', '@loc', '@name', '@origName', '@topModule', 'var', 'instance'])
odict_keys(['@fl', '@loc', '@name', '@origName', 'var'])
odict_keys(['@fl', '@loc', '@name', '@origName', 'var'])


## top 모듈인 경우 vs top 모듈이 아닌 경우

module은 기본적으로

```
module
|- '@fl'
|- '@loc'
|- '@name'
|- '@origName'
|- 'var'      
|- '@topModule'    # top module인 경우
|- 'instance'      # 내부에 사용된 instance가 있는 경우
```

top module이면 '@topModule' 이라는 dictionary 내부 값을 가집니다.  

module 내부에서 외부 module을 사용한 경우는 instance를 가집니다. 

그 외에 

- '@fl'
- '@loc'
- '@name'
- '@origName'
- 'var' 

위 값들은 기본적으로 가지는 값들 입니다. 



In [28]:
top = None
for i in info['verilator_xml']['netlist']['module']:
    if('@topModule' in i.keys()):
        top = i       

In [29]:
print(type(top['instance']))
print(len(top['instance']))
print(top.keys())

<class 'list'>
7
odict_keys(['@fl', '@loc', '@name', '@origName', '@topModule', 'var', 'instance'])


### port는 'var'에 저장 되어 있습니다.

## internal port와 external port의 차이

dictionary 항목으로 @dir을 가지고 있다면 external port 입니다. 

In [30]:
top_dict = {}
top_dict['name']=None
top_dict['external_port']=None
top_dict['internal_port']=None
top_dict['instance']=None

print(len(top['var']))
for i in top['var']:
    print(i.keys())
    

20
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@dir', '@vartype', '@origName'])
odict_keys(['@fl', '@loc', '@name', '@dtype_id', '@vartype', '@origName'])
odict_key

In [31]:
print(len(top['var']))
for i in top['var']:
    print(i['@name'],end= " ")
    print(i['@dtype_id'])

20
i_clock 1
i_resetn 1
o_test_0_valid 1
i_test_0_ready 1
o_test_0_wdata 2
i_test_0_rdata 2
o_test_2_valid 1
o_test_2_wdata 2
i_test_2_rdata 2
o_test_3_wdata 3
i_test_3_rdata 3
test_1_valid 1
test_1_ready 1
test_1_wdata 2
test_1_rdata 2
test_2_valid 1
test_2_ready 1
test_2_wdata 2
test_2_rdata 2
tmp 3


# instance를 살펴보기



In [32]:
print(len(top['instance']))
for i in top['instance']:
    print(i.keys())

7
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])


### instance는 

### ['@fl', '@loc', '@name', '@defName', '@origName', 'port']

### 이렇게 가지고 있습니다.


## 우리가 필요한건 instance의 port 정보 입니다. 

---


## instance가 가진 port는 아래와 같은 종류가 있습니다.

- 연결 됨 : odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])  
- 연결 안됨 : odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex'])  
- 포트 병합으로 연결됨 : odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'concat'])  
- {}를 사용해서 연결됨 : odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'replicate'])  


In [33]:
print(len(top['instance']))
for i in top['instance']:
    print(i.keys())
    for j in i['port']:
        print(j.keys())
    

7
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@defName', '@origName', 'port'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name', '@direction', '@portIndex', 'varref'])
odict_keys(['@fl', '@loc', '@name

## port의 const에서 원하는 정보 추출

아래 함수를 이용해서 const일경우 원하는 정보를 추출합니다. 

아직은 모든 경우의 수를 관리하는 모습이 아니지만 점차 가다듬어 나간다면  
모든 경우의 수를 다룰 수 있을것이라 생각합니다. 

In [34]:
def port_const_extract(file_dict, port_info):
    
    try:
        #concat
        l_id=port_info[0]['@loc'].split(',')[0]
        l_s_y = int(port_info[0]['@loc'].split(',')[1])-1
        l_s_x = int(port_info[0]['@loc'].split(',')[2])-1
        l_e_y = int(port_info[0]['@loc'].split(',')[3])-1
        l_e_x = int(port_info[0]['@loc'].split(',')[4])-1

        m_id =port_info[1]['@loc'].split(',')[0]
        m_s_y=(int(port_info[1]['@loc'].split(',')[1])-1 )
        m_s_x=(int(port_info[1]['@loc'].split(',')[2])-1 )
        m_e_y=(int(port_info[1]['@loc'].split(',')[3])-1 )
        m_e_x=(int(port_info[1]['@loc'].split(',')[4])-1 )

        result = [ file_dict[m_id][m_s_y][m_s_x:m_e_x],file_dict[l_id][l_s_y][l_s_x:l_e_x] ]
    except:
        #replicate
        f_id =port_info['@loc'].split(',')[0]
        s_y=(int(port_info['@loc'].split(',')[1])-1 )
        s_x=(int(port_info['@loc'].split(',')[2])-1 )
        e_y=(int(port_info['@loc'].split(',')[3])-1 )
        e_x=(int(port_info['@loc'].split(',')[4])-1 )
        
        result = int(file_dict[f_id][s_y][s_x:e_x])
        
    
    return result
    
    

## 재귀 함수를 이용한 port 정보의 추출

**Concat** 을 사용하여 port를 생성한 경우  

마지막 포트를 만날때까지 지속적으로 하위 concat이 발생합니다.   
즉 port 개수가 5개라면 4개의 concat을 풀어야 한다는 것입니다. 

따라서 아래와 같이 **재귀 함수**를 이용해서 port정보를 추출합니다. 

In [35]:
def concat_port_extract(file_dict, concat_port):
    try:
        print(concat_port['varref']['@name'], end = ", ")
        print(concat_port['sel']['varref']['@name'],end=" ")
        print(port_const_extract(file_dict, concat_port['sel']['const']), end=", ")
    except:
        print(concat_port['sel']['varref']['@name'],end=" ")
        print(port_const_extract(file_dict, concat_port['sel']['const']), end=", ")
    
    try:
        concat_port_extract(file_dict,concat_port['concat'])
    except:
        pass

## 실제 instance의 값을 추출하는 코드

아래 코드를 활용해서 top이 가진 instance의 이름과 port 연결 정보를 가져옵니다.  

In [36]:
for i in top['instance']:
    print(i['@name'])
    for j in i['port']:
        if( 'varref' in j.keys()):
            print("(",j['@portIndex'],")", j['@name'],"<=",j['varref']['@name'])
            pass
        elif('concat' in j.keys()):
            print("(",j['@portIndex'],")",j['@name'],"<=", end=" ")
                                    
            #-----------------------------------
            concat_port_extract(code,j['concat'])
            #-----------------------------------
            print("")
            
        
        
        elif('replicate' in j.keys()):
            print("(",j['@portIndex'],")",j['@name'],"<=",end=" ")
            repli = port_const_extract(code, j['replicate']['const'])
            for r in range(repli):
                print((j['replicate']['varref']['@name']) ,end=", ")
            print("")
            
        
        else:
            print("(",j['@portIndex'],")",j['@name'],"<="," ")

    print("\n\n")
            
            

u_master_0
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= o_test_0_valid
( 4 ) i_ready <= i_test_0_ready
( 5 ) o_data <= o_test_0_wdata
( 6 ) i_data <= i_test_0_rdata



u_master_1
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= test_1_valid
( 4 ) i_ready <= test_1_ready
( 5 ) o_data <= test_1_wdata
( 6 ) i_data <= test_1_rdata



u_slave_1
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) i_valid <= test_1_valid
( 4 ) o_ready <= test_1_ready
( 5 ) i_data <= test_1_wdata
( 6 ) o_data <= test_1_rdata



u_master_2
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= o_test_2_valid
( 4 ) i_ready <=  
( 5 ) o_data <= o_test_2_wdata
( 6 ) i_data <= i_test_2_rdata



u_slave_2
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) i_valid <= o_test_2_valid
( 4 ) o_ready <=  
( 5 ) i_data <= o_test_2_wdata
( 6 ) o_data <=  



u_master_3
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <=  
( 4 ) i_ready <=  
( 5 ) o_data <= o_te

In [37]:
info['verilator_xml']['netlist']['module'][0]

OrderedDict([('@fl', 'c1'),
             ('@loc', 'c,1,8,1,23'),
             ('@name', 'connection_test'),
             ('@origName', 'connection_test'),
             ('@topModule', '1'),
             ('var',
              [OrderedDict([('@fl', 'c2'),
                            ('@loc', 'c,2,16,2,23'),
                            ('@name', 'i_clock'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_clock')]),
               OrderedDict([('@fl', 'c3'),
                            ('@loc', 'c,3,16,3,24'),
                            ('@name', 'i_resetn'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_resetn')]),
               OrderedDict([('@fl', 'c4'),
                            ('@loc', 'c,4,24,4,38

In [38]:
info['verilator_xml']['netlist']['module'][1]

OrderedDict([('@fl', 'c108'),
             ('@loc', 'c,108,8,108,14'),
             ('@name', 'master'),
             ('@origName', 'master'),
             ('var',
              [OrderedDict([('@fl', 'c109'),
                            ('@loc', 'c,109,20,109,25'),
                            ('@name', 'i_clk'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_clk')]),
               OrderedDict([('@fl', 'c110'),
                            ('@loc', 'c,110,20,110,26'),
                            ('@name', 'i_rstn'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_rstn')]),
               OrderedDict([('@fl', 'c111'),
                            ('@loc', 'c,111,20,111,27'),
                            ('@n

In [39]:
info['verilator_xml']['netlist']['module'][2]

OrderedDict([('@fl', 'c119'),
             ('@loc', 'c,119,8,119,13'),
             ('@name', 'slave'),
             ('@origName', 'slave'),
             ('var',
              [OrderedDict([('@fl', 'c120'),
                            ('@loc', 'c,120,20,120,25'),
                            ('@name', 'i_clk'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_clk')]),
               OrderedDict([('@fl', 'c121'),
                            ('@loc', 'c,121,20,121,26'),
                            ('@name', 'i_rstn'),
                            ('@dtype_id', '1'),
                            ('@dir', 'input'),
                            ('@vartype', 'logic'),
                            ('@origName', 'i_rstn')]),
               OrderedDict([('@fl', 'c122'),
                            ('@loc', 'c,122,20,122,27'),
                            ('@nam

In [40]:
info['verilator_xml']['netlist']['typetable']

OrderedDict([('@fl', 'a0'),
             ('@loc', 'a,0,0,0,0'),
             ('basicdtype',
              [OrderedDict([('@fl', 'c120'),
                            ('@loc', 'c,120,20,120,25'),
                            ('@id', '1'),
                            ('@name', 'logic')]),
               OrderedDict([('@fl', 'c6'),
                            ('@loc', 'c,6,17,6,18'),
                            ('@id', '2'),
                            ('@name', 'logic'),
                            ('@left', '31'),
                            ('@right', '0')]),
               OrderedDict([('@fl', 'c13'),
                            ('@loc', 'c,13,17,13,18'),
                            ('@id', '3'),
                            ('@name', 'logic'),
                            ('@left', '15'),
                            ('@right', '0')]),
               OrderedDict([('@fl', 'c90'),
                            ('@loc', 'c,90,33,90,34'),
                            ('@id', '4'),
              

## port type table 생성

Verilator json은 port type의 table을 통해서 port의 형태를 보여줍니다.  

port_type 이라는 dictionary 형태의 변수를 선언해서 그 안에 port_type을 담아줍니다.

In [41]:
type_table = info['verilator_xml']['netlist']['typetable']['basicdtype'];
port_type = {}
for i in type_table:
    if('@left' in i.keys()):
        port_type[i['@id']] = {'msb':int(i['@left']), 'lsb':int(i['@right'])}
    else:
        port_type[i['@id']] = {'msb':0,'lsb':0}

In [42]:
port_type

{'1': {'msb': 0, 'lsb': 0},
 '2': {'msb': 31, 'lsb': 0},
 '3': {'msb': 15, 'lsb': 0},
 '4': {'msb': 7, 'lsb': 0},
 '5': {'msb': 3, 'lsb': 0},
 '6': {'msb': 23, 'lsb': 0},
 '7': {'msb': 31, 'lsb': 0}}

## 이제 port type을 이용해서 앞서 만들어둔 port들을 좀더 잘 표현해 보겠습니다. 

In [43]:
print(len(top['var']))
for i in top['var']:
    print(i['@name'],end= " ")
    print(port_type[i['@dtype_id']])

20
i_clock {'msb': 0, 'lsb': 0}
i_resetn {'msb': 0, 'lsb': 0}
o_test_0_valid {'msb': 0, 'lsb': 0}
i_test_0_ready {'msb': 0, 'lsb': 0}
o_test_0_wdata {'msb': 31, 'lsb': 0}
i_test_0_rdata {'msb': 31, 'lsb': 0}
o_test_2_valid {'msb': 0, 'lsb': 0}
o_test_2_wdata {'msb': 31, 'lsb': 0}
i_test_2_rdata {'msb': 31, 'lsb': 0}
o_test_3_wdata {'msb': 15, 'lsb': 0}
i_test_3_rdata {'msb': 15, 'lsb': 0}
test_1_valid {'msb': 0, 'lsb': 0}
test_1_ready {'msb': 0, 'lsb': 0}
test_1_wdata {'msb': 31, 'lsb': 0}
test_1_rdata {'msb': 31, 'lsb': 0}
test_2_valid {'msb': 0, 'lsb': 0}
test_2_ready {'msb': 0, 'lsb': 0}
test_2_wdata {'msb': 31, 'lsb': 0}
test_2_rdata {'msb': 31, 'lsb': 0}
tmp {'msb': 15, 'lsb': 0}


## 위에서 만든 top module의 내부 instance 값도 같이 표현해보겠습니다. 

In [44]:
for i in top['instance']:
    print(i['@name'])
    for j in i['port']:
        if( 'varref' in j.keys()):
            print("(",j['@portIndex'],")", j['@name'],"<=",j['varref']['@name'])
            pass
        elif('concat' in j.keys()):
            print("(",j['@portIndex'],")",j['@name'],"<=", end=" ")
                                    
            #-----------------------------------
            concat_port_extract(code,j['concat'])
            #-----------------------------------
            print("")
            
        
        
        elif('replicate' in j.keys()):
            print("(",j['@portIndex'],")",j['@name'],"<=",end=" ")
            repli = port_const_extract(code, j['replicate']['const'])
            for r in range(repli):
                print((j['replicate']['varref']['@name']) ,end=", ")
            print("")
            
        
        else:
            print("(",j['@portIndex'],")",j['@name'],"<="," ")

    print("\n\n")

u_master_0
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= o_test_0_valid
( 4 ) i_ready <= i_test_0_ready
( 5 ) o_data <= o_test_0_wdata
( 6 ) i_data <= i_test_0_rdata



u_master_1
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= test_1_valid
( 4 ) i_ready <= test_1_ready
( 5 ) o_data <= test_1_wdata
( 6 ) i_data <= test_1_rdata



u_slave_1
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) i_valid <= test_1_valid
( 4 ) o_ready <= test_1_ready
( 5 ) i_data <= test_1_wdata
( 6 ) o_data <= test_1_rdata



u_master_2
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <= o_test_2_valid
( 4 ) i_ready <=  
( 5 ) o_data <= o_test_2_wdata
( 6 ) i_data <= i_test_2_rdata



u_slave_2
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) i_valid <= o_test_2_valid
( 4 ) o_ready <=  
( 5 ) i_data <= o_test_2_wdata
( 6 ) o_data <=  



u_master_3
( 1 ) i_clk <= i_clock
( 2 ) i_rstn <= i_resetn
( 3 ) o_valid <=  
( 4 ) i_ready <=  
( 5 ) o_data <= o_te

## 이제 두가지 출력을 합치면 top module의 입력 출력에 대한 정의가 마무리 됩니다. 

좀 더 자세히 혹은 편리하게 사용하고자 한다면 위 코드를 가다듬어서 진행해야 합니다. 

본 jupyter notebook을 통해서 개발 과정을 남기도록 하겠습니다. 